<a href="https://colab.research.google.com/github/Mekondjo-EDSA/dsi_repo/blob/master/Twist_challenge_bikes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## OVERVIEW 
This notebook shows how to use BigQuery to train a model using SQL

## SETUP


Set up your GCP project

**The following steps are required, regardless of your notebook environment.**

1. [Select or create a GCP project.](https://console.cloud.google.com/cloud-resource-manager)
2. [Enable the BigQuery Storage API](https://cloud.google.com/bigquery/docs/reference/storage/#enabling_the_api)
3. Enter your project ID in the cell below. Then run the  cell to make sure the
Cloud SDK uses the right project for all the commands in this notebook.


import required modules

In [ ]:

import pandas as pd 
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from matplotlib import pyplot as plt
from google.cloud import bigquery

Install required Packages, and restart runtime

In [ ]:
!pip install pip --upgrade 'google-cloud-bigquery[bqstorage,pandas]'


In [ ]:
%load_ext google.cloud.bigquery

In [ ]:
%load_ext google.colab.data_table

Authenticate 

In [ ]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


Set project ID and list public datasets available on BigQuery

In [ ]:
#variables
project_id = 'bigquery-public-data'
client = bigquery.Client(project=project_id)
for dataset in client.list_datasets():
  print(dataset.dataset_id)

Run some queries to get a feel for the data and how to structure the code

In [ ]:
%%bigquery --project twist-project-343915

SELECT
start_station_id, start_station_name, round(AVG(duration),2)duration
FROM `bigquery-public-data.london_bicycles.cycle_hire`
group by 1,2

In [ ]:
%%bigquery --project twist-project-343915

SELECT
EXTRACT(dayofweek from start_date)dayofweek, round(AVG(duration),2)duration
FROM `bigquery-public-data.london_bicycles.cycle_hire`
group by 1

In [ ]:
%%bigquery --project twist-project-343915

SELECT
EXTRACT(hour from start_date)hourofday, round(AVG(duration),2)duration
FROM `bigquery-public-data.london_bicycles.cycle_hire`
group by 1

In [ ]:
%%bigquery --project twist-project-343915

SELECT
bikes_count, AVG(duration) as duration
FROM 
  `bigquery-public-data`.london_bicycles.cycle_hire 
JOIN 
  `bigquery-public-data`.london_bicycles.cycle_stations
ON 
  cycle_hire.start_station_name = cycle_stations.name
group by 1

The following section of code should be run in your Bigquery project in order to create the dataset and model.

In [ ]:
#create dataset 
%%bigquery --project twist-project-343915

SELECT
duration, 
start_station_name,
CAST(EXTRACT(dayofweek from start_date) as string)dayofweek,
CAST(EXTRACT(hour from start_date)as string)hourofday
FROM `bigquery-public-data.london_bicycles.cycle_hire`

In [ ]:
#create and train model
%%bigquery --project twist-project-343915

CREATE OR REPLACE MODEL
bike_model.model
OPTIONS(input_label_cols = ['duration'], model_type = 'linear_reg') AS 
SELECT
duration, 
start_station_name,
CAST(EXTRACT(dayofweek from start_date) as string)dayofweek,
CAST(EXTRACT(hour from start_date)as string)hourofday
FROM `bigquery-public-data.london_bicycles.cycle_hire`

In [ ]:
#attempts to improve model performance 
%%bigquery --project twist-project-343915

CREATE OR REPLACE MODEL
bike_model.model_bucket
OPTIONS(input_label_cols = ['duration'], model_type = 'linear_reg') AS 
SELECT
duration, 
start_station_name,
(EXTRACT(dayofweek from start_date) BETWEEN 2 AND 6, 'weekday', 'weekend') dayofweek,
ML.BUCKETIZE(EXTRACT(hour from start_date),[5,10,17]) hourofday
FROM `bigquery-public-data.london_bicycles.cycle_hire`

## Evaluate
Evaluation of the model can be performed in BigQuery or in Colab

In [25]:
#Evaluate model
%%bigquery --project twist-project-343915

SELECT * FROM ML.EVALUATE (MODEL `bike_model.model`)

,mean_absolute_error,mean_squared_error,mean_squared_log_error,median_absolute_error,r2_score,explained_variance
0,1025.364041,1.862628e+08,0.862452,542.246965,0.003625,0.003646


## Predict

In [28]:
#make predictions 
%%bigquery --project twist-project-343915

SELECT 
*
FROM 
ML.PREDICT (MODEL bike_model.model_bucket,
  (
  SELECT
    start_station_name, 
    start_date 
    FROM
      `bigquery-public-data.london_bicycles.cycle_hire`
  LIMIT
    100))  

,predicted_duration,start_station_name,start_date
0,1823.906069,"Serpentine Car Park, Hyde Park",2015-09-03 11:52:00+00:00
1,3649.442024,"Speakers' Corner 1, Hyde Park",2015-08-16 09:56:00+00:00
2,878.984631,"Queen Street 2, Bank",2017-06-09 17:56:00+00:00
3,1926.294259,"Teviot Street, Poplar",2017-04-22 09:36:00+00:00
4,2555.515019,"Wandsworth Rd, Isley Court, Wandsworth Road",2016-10-09 03:52:00+00:00
...,...,...,...
95,1573.869099,"Whiteley's, Bayswater",2016-08-29 19:29:00+00:00
96,1033.576282,"Belgrave Road, Victoria",2016-03-16 07:45:00+00:00
97,1069.852917,"Cartwright Gardens , Bloomsbury",2015-06-10 14:29:00+00:00
98,1350.474924,"Warwick Avenue Station, Maida Vale",2015-01-21 05:14:00+00:00
